In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [3]:
data=pd.read_csv("Color.txt", sep="\t")

In [4]:
data.isnull().sum(axis=0)

X1    0
X2    0
X3    0
Y     0
dtype: int64

In [5]:
data=np.matrix(data)

In [6]:
#Min max normalize
#X is data set to normalize, X_tr is data set to normalize based on
# def norm(X,X_tr):
#     X_tr_mean=X_tr.mean(0)
#     X_tr_std=X_tr.std(0)
#     X=(X-X_tr_mean)/X_tr_std
#     return(X)
def norm(X,X_tr):
    X_tr_min=X_tr.min(0)
    X_tr_max=X_tr.max(0)
    X=(X-X_tr_min)/(X_tr_max-X_tr_min)
    return(X)

In [7]:
#Just a test of normalizer, doing min max
norm(data,data)

matrix([[0.29019608, 0.33333333, 0.48235294, 0.        ],
        [0.28627451, 0.32941176, 0.47843137, 0.        ],
        [0.28235294, 0.3254902 , 0.4745098 , 0.        ],
        ...,
        [0.63921569, 0.63529412, 0.43921569, 1.        ],
        [0.63921569, 0.63529412, 0.43921569, 1.        ],
        [1.        , 1.        , 1.        , 1.        ]])

In [8]:
#Shuffle data
np.random.shuffle(data)

In [9]:
k=10
data.shape[0]

245057

In [10]:
test=data[:(int(data.shape[0]/15)),:]
train=data[(int(data.shape[0]/15)):,:]

In [11]:
import gc
del data
gc.collect()

7

In [12]:
test.shape[0]+train.shape[0]

245057

In [13]:
data_sets=np.array_split(train,k)

In [14]:
def phi(X,y):
    rows, cols=np.where(y[:,0]==1)
    ones=X[rows,:]
    center=ones.mean(0)


    #Using the covariance matrix inverse to find the 
    #approximate dimensions of the ellipsoid
    A=np.matrix(np.linalg.inv(np.cov(X.T)))
    #Do the transformation
    phi_x=np.multiply((X-center),(A@(X-center).T).T)
    phi_x=np.sum(phi_x,axis=1)-1
    phi_x=np.hstack((np.ones((X.shape[0],1)),phi_x))
    return phi_x

In [21]:
epochs=2000
eta=.00001
lam=0.001
accuracy=[]
recall=[]
for j in range(k):
    val=data_sets[j]
    train=np.vstack(data_sets[:j]+data_sets[(j+1):])

    X_val=norm(val[:,:-1],train[:,:-1])
    y_val=val[:,-1]
    #want skin as positive, currently not skin is, so change it
    y_val=(y_val-2)*-1
    phi_val=phi(X_val,y_val)

    #normalize train and add bias column
    X_train=norm(train[:,:-1],train[:,:-1])
    y_train=train[:,-1]
    y_train=(y_train-2)*-1
    phi_tr=phi(X_train,y_train)
    #initialize weights for each fold
    w=np.random.randn(phi_tr.shape[1],1)
    for i in range(epochs):
        p=1/(1+np.exp(-phi_tr@w))
        w=w-eta*phi_tr.T@(p-y_train)-lam*w
    
    p_val=1/(1+np.exp(-phi_val@w))
    y_pred=np.rint(p_val)
    accuracy.append((y_pred == y_val).mean()*100)

    recall.append((y_pred[(y_pred==y_val) & (y_pred==1)].shape[1]/y_val[y_val==1].shape[1]))
    

In [22]:
np.mean(accuracy)

98.24720181881776

In [23]:
np.mean(recall)

0.9691270757098904

In [18]:
train=np.vstack(data_sets)

X_test=norm(test[:,:-1],train[:,:-1])
y_test=test[:,-1]
#want skin as positive, currently not skin is, so change it
y_test=(y_test-2)*-1
phi_test=phi(X_test,y_test)

#normalize train and add bias column
X_train=norm(train[:,:-1],train[:,:-1])
y_train=train[:,-1]
y_train=(y_train-2)*-1
phi_tr=phi(X_train,y_train)
#initialize weights for each fold
w=np.random.randn(phi_tr.shape[1],1)
for i in range(epochs):
    p=1/(1+np.exp(-phi_tr@w))
    w=w-eta*phi_tr.T@(p-y_train)-lam*w
p_test=1/(1+np.exp(-phi_test@w))
y_pred=np.rint(p_test)
accuracy=(y_pred == y_test).mean()*100
recall=y_pred[(y_pred==y_test) & (y_pred==1)].shape[1]/y_test[y_test==1].shape[1]

In [19]:
accuracy

98.37179408704169

In [20]:
recall

0.9660666863381528